# Introduction

Hello people, welcome to this kernel. In this kernel I am going to classify emotions using their texts. This kernel will be short as much as it can, so you can examine everything in a short time. Let's start.

# Notebook Content
1. Importing Libraries and The Data
1. Preparing The Dataset
    * Splitting X and Y
    * Converting Y To Integer
1. Tokenization
    * Tokenizing
    * Padding
1. Modeling and Predicting
1. Conclusion




In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions-dataset-for-nlp/test.txt
/kaggle/input/emotions-dataset-for-nlp/train.txt
/kaggle/input/emotions-dataset-for-nlp/val.txt


In [2]:
test = pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/test.txt',sep=";",header=None)
train = pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/train.txt',sep=";",header=None)
val = pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/val.txt',sep=";",header=None)

In [3]:
train.head()

,0,1
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [4]:
test.head()

,0,1
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [5]:
val.head()

,0,1
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       16000 non-null  object
 1   1       16000 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2000 non-null   object
 1   1       2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [8]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2000 non-null   object
 1   1       2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


# Preparing The Dataset

In [9]:
# Splitting X and Y
x_train = train.iloc[:,0] 
y_train = train.iloc[:,1] 

x_test = test.iloc[:,0] 
y_test = test.iloc[:,1] 

x_val = test.iloc[:,0] 
y_val = test.iloc[:,1] 

x_train,y_train = np.array(x_train),np.array(y_train)
x_test,y_test = np.array(x_test),np.array(y_test)
x_val,y_val = np.array(x_val),np.array(y_val)

print(x_train.shape,"|",y_train.shape)
print(x_test.shape,"|",y_test.shape)
print(x_val.shape,"|",y_val.shape)

(16000,) | (16000,)
(2000,) | (2000,)
(2000,) | (2000,)


* Now I will convert y to integers.

In [10]:
print(train.iloc[:,1].value_counts())

int_y_train = []
int_y_test = []
int_y_val = []

for l in y_train:
    
    if l == "joy":        
        int_y_train.append(0)
        
    if l == "sadness":       
        int_y_train.append(1)            
    if l == "anger":      
        int_y_train.append(2)      
    if l == "fear":
        int_y_train.append(3)
    if l == "love":
        int_y_train.append(4)
    if l == "surprise":
        int_y_train.append(5)
        
        
for l in y_test:
    
    if l == "joy":        
        int_y_test.append(0)
        
    if l == "sadness":       
        int_y_test.append(1)            
    if l == "anger":      
        int_y_test.append(2)      
    if l == "fear":
        int_y_test.append(3)
    if l == "love":
        int_y_test.append(4)
    if l == "surprise":
        int_y_test.append(5)
        
for l in y_val:
    
    if l == "joy":        
        int_y_val.append(0)
        
    if l == "sadness":       
        int_y_val.append(1)            
    if l == "anger":      
        int_y_val.append(2)      
    if l == "fear":
        int_y_val.append(3)
    if l == "love":
        int_y_val.append(4)
    if l == "surprise":
        int_y_val.append(5)
        


        
int_y_train,int_y_test,int_y_val = np.array(int_y_train),np.array(int_y_test),np.array(int_y_val)
from sklearn import preprocessing
from keras.utils import np_utils

le = preprocessing.LabelEncoder()
le.fit(int_y_train)

encoded_y_train = le.transform(int_y_train)
encoded_y_test = le.transform(int_y_test)
encoded_y_val = le.transform(int_y_val)

encoded_y_train = np_utils.to_categorical(encoded_y_train)
encoded_y_test = np_utils.to_categorical(encoded_y_test)
encoded_y_val = np_utils.to_categorical(encoded_y_val)

print(encoded_y_train)

print(int_y_train[:10])

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: 1, dtype: int64
[[0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
[1 1 2 4 2 1 5 3 0 4]


# Tokenization

In this section I am going to tokenize all the texts and prepare for deep learning.

In [11]:
x_train[0]

'i didnt feel humiliated'

In [12]:
x_test[1]

'im updating my blog because i feel shitty'

In [13]:
x_val[1]

'im updating my blog because i feel shitty'

* Our texts looks great, we can tokenize them directly

In [14]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

x_train_cl = []
x_test_cl = []
x_val_cl = []


# Deleting stopwords
for text in x_train:
    
    text = text.split()
    text = [word for word in text if word not in stopwords]
    text = " ".join(text)
    x_train_cl.append(text)
    
for text in x_test:
    
    text = text.split()
    text = [word for word in text if word not in stopwords]
    text = " ".join(text)
    x_test_cl.append(text)

for text in x_val:
    
    text = text.split()
    text = [word for word in text if word not in stopwords]
    text = " ".join(text)
    x_val_cl.append(text)
    
x_train,x_test,x_val = np.array(x_train_cl),np.array(x_test_cl),np.array(x_val_cl)

# We use total_text for fitting tokenizer in general 
total_text = np.concatenate((x_train,x_test,x_val),axis=0)

num_words = 2000

tokenizer = Tokenizer(num_words = num_words) 

tokenizer.fit_on_texts(total_text)

list(tokenizer.word_index)[:10]

['feel',
 'feeling',
 'like',
 'im',
 'really',
 'know',
 'time',
 'get',
 'little',
 'people']

In [15]:
# Tokenizing everything

x_train_token = tokenizer.texts_to_sequences(x_train)
x_test_token = tokenizer.texts_to_sequences(x_test)
x_val_token = tokenizer.texts_to_sequences(x_val)

total_token = np.concatenate((x_train_token,x_test_token,x_val_token),axis=0)

In [16]:
# Padding
print(np.mean([len(text) for text in total_token]))

7.6699


* We can use 20 as padding.

In [17]:
x_train_pad = pad_sequences(x_train_token,20) 
x_test_pad = pad_sequences(x_test_token,20)
x_val_pad = pad_sequences(x_val_token,20)

print(x_train_pad[0],end="\n-------------------------------------------------------------------------\n")
print(x_train_pad[1])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  50
   1 495]
-------------------------------------------------------------------------
[   0    0    0    0    0    0    0    0    0    0    0    0   30    2
  464  425   44   54 1573 1304]


# Modeling and Predicting

Our data is ready, now lets build a simple RNN model.

In [18]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,CuDNNGRU,Embedding,Bidirectional


model = Sequential()

model.add(Embedding(input_dim=2000
                   ,output_dim=100
                   ,input_length=20))

model.add(Bidirectional(CuDNNGRU(units=16,return_sequences=True)))

model.add(Bidirectional(CuDNNGRU(units=8)))

model.add(Dense(6,activation="softmax"))

model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 20, 32)            11328     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 16)                2016      
_________________________________________________________________
dense (Dense)                (None, 6)                 102       
Total params: 213,446
Trainable params: 213,446
Non-trainable params: 0
_________________________________________________________________


In [19]:
print(x_train_pad.shape)
print(y_train.shape)
model.fit(x_train_pad,encoded_y_train,epochs=10,batch_size=20)

(16000, 20)
(16000,)
Epoch 1/10
800/800 [==============================] - 7s 9ms/step - loss: 1.1128 - accuracy: 0.5868
Epoch 2/10
800/800 [==============================] - 7s 9ms/step - loss: 0.3845 - accuracy: 0.8746
Epoch 3/10
800/800 [==============================] - 7s 8ms/step - loss: 0.2337 - accuracy: 0.9134
Epoch 4/10
800/800 [==============================] - 7s 9ms/step - loss: 0.1857 - accuracy: 0.9289
Epoch 5/10
800/800 [==============================] - 6s 8ms/step - loss: 0.1551 - accuracy: 0.9382
Epoch 6/10
800/800 [==============================] - 7s 8ms/step - loss: 0.1367 - accuracy: 0.9449
Epoch 7/10
800/800 [==============================] - 7s 8ms/step - loss: 0.1248 - accuracy: 0.9504
Epoch 8/10
800/800 [==============================] - 7s 8ms/step - loss: 0.1143 - accuracy: 0.9542
Epoch 9/10
800/800 [==============================] - 7s 9ms/step - loss: 0.1064 - accuracy: 0.9584
Epoch 10/10
800/800 [==============================] - 7s 9ms/step - loss: 0.09

In [20]:
preds = model.predict_classes(x_test_pad)
from sklearn.metrics import accuracy_score

accuracy_score(preds,int_y_test)

0.9205

# Conclusion

Now lets take a look at what did I do:

1. First I imported the data.
1. Then I've prepared labels
    * object to int
    * int to one hot encode
1. Then I've prepared texts
    * tokenizing
    * padding
1. Finally I've build the model.


If you like this kernel, please upvote. If you have questions in your mind, I am wating for them. 